# CHAPTER 5 - QAOA: Quantum Approximate Optimization Algorithm - PennyLane Code

*Note*: You may skip the following cell if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install pennylane==0.26.0

In [ ]:
import pennylane as qml
from pennylane import PauliZ
coefficients = [2,-1,3.5]
paulis = [PauliZ(0)@PauliZ(1),PauliZ(0)@PauliZ(2),PauliZ(1)]
H = qml.Hamiltonian(coefficients,paulis)
print(H)

In [ ]:
print(qml.matrix(H))

In [ ]:
from pennylane.utils import sparse_hamiltonian
print(sparse_hamiltonian(H))

In [ ]:
H = 2*PauliZ(0)@PauliZ(1) - PauliZ(0)@PauliZ(2) +3.5*PauliZ(1)


In [ ]:
from pennylane import qaoa

H0 = qml.PauliX(0) + qml.PauliX(1)
H1 = 1.0*qml.PauliZ(0) @ qml.PauliZ(1) 

wires = range(2)
dev = qml.device("default.qubit", wires=wires)

p = 2

@qml.qnode(dev)
def energy(angles):
    for w in wires:
        qml.Hadamard(wires=w)
    for i in range(p):
        qaoa.cost_layer(angles[2*i+1], H1)
        qaoa.mixer_layer(angles[2*i], H0)
    return qml.expval(H1)

In [ ]:
from pennylane import numpy as np
optimizer = qml.GradientDescentOptimizer()
steps = 20
angles = np.array([1,1,1,1], requires_grad=True)

for i in range(steps):
    angles = optimizer.step(energy, angles)

print("Optimal angles", angles)

In [ ]:
@qml.qnode(dev)
def sample_solutions(angles):
    for w in wires:
        qml.Hadamard(wires=w)
    for i in range(p):
        qaoa.cost_layer(angles[2*i+1], H1)
        qaoa.mixer_layer(angles[2*i], H0)
    return qml.sample()
print(sample_solutions(angles, shots = 5))